### ПОСТАНОВКА ЗАДАЧИ

Современные маркетинговые стратегии ориентированы на персонализацию взаимодействия с клиентами и повышение эффективности рекламных кампаний. Компания располагает историческими данными о клиентах, включающими демографические характеристики, уровень дохода, покупательское поведение и информацию об участии в предыдущих маркетинговых кампаниях.

Основной бизнес-проблемой является низкий уровень отклика клиентов на маркетинговые предложения, что приводит к неэффективным затратам на коммуникации и снижению возврата инвестиций в маркетинг.

В связи с этим возникает задача анализа клиентской базы и выявления факторов, влияющих на вероятность отклика клиента на маркетинговую кампанию.

### Цель работы

Разработать подход к анализу и моделированию клиентского поведения, позволяющий прогнозировать отклик клиента на маркетинговое предложение на основе его характеристик и истории взаимодействия с компанией.

### Основные задачи исследования

1. Провести исследовательский анализ данных (EDA) для изучения структуры клиентской базы и выявления закономерностей покупательского поведения.
2. Выполнить очистку и подготовку данных для аналитического и моделирующего этапов.
3. Сформировать новые информативные признаки, отражающие уровень активности и ценность клиента.
4. Определить ключевые факторы, влияющие на отклик клиентов.
5. Построить модель, прогнозирующую вероятность отклика клиента на маркетинговую кампанию.
6. Оценить качество модели с использованием метрик классификации.
7. Сформулировать практические рекомендации по повышению эффективности маркетинговых кампаний.

In [ ]:
# импортируем необходимые модули
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import integrate, optimize
from sklearn import preprocessing, model_selection, metrics
from sklearn.cluster import KMeans
# from kneed import KneeLocator - у меня не работает 
from IPython.display import display
%matplotlib inline


In [ ]:
# загружаем данные из файла в формате CSV, указываем символ-разделитель и сохраняем данные в дата-фрейм (переменную df)
df = pd.read_csv('../data/marketing_campaign.csv',sep='\t')
# проверяем, что данные загрузились - выводим первые несколько строк таблицы
df.head()

## Признаки:
- ID - уникальный идентификатор клиента 
- Year_Birth - год рождения клиента 
- Education - уровень образования  
- Marital_Status - семейное положение  
- Income - годовой доход семьи клиента 
- Kidhome - количество детей до 12 лет в семье
- Teenhome - количество подростков 13-18 лет в семье
- Dt_Customer - дата регистрации клиента
- Recency - количество дней с момента последней покупки
- MntWines - траты на вино
- MntFruits - траты на фрукты
- MntMeatProducts - траты на мясо
- MntFishProducts - траты на рыбу
- MntSweetProducts - траты на сладости
- MntGoldProds - траты на золото (ювелирные изделия)
- NumDealsPurchases - покупки со скидкой
- NumWebPurchases - покупки через сайт
- NumCatalogPurchases - покупки по каталогу
- NumStorePurchases - покупки в магазине
- NumWebVisitsMonth - количество посещений сайта в месяц
- AcceptedCmp1 - участие в 1-й кампании 
- AcceptedCmp2 - участие во 2-й кампании 
- AcceptedCmp3 - участие в 3-й кампании 
- AcceptedCmp4 - участие в 4-й кампании 
- AcceptedCmp5 - участие в 5-й кампании 
- Complain - подавал ли жалобу за последние 2 года 
- Z_CostContact - техническая колонка 
- Z_Revenue - техническая колонка 
- Response - отклик на предложение в последней кампании (1 - согласился, 0 - отказался)

### Информация о датасете

In [ ]:
# целевой признак Response

# объём данных (строк и столбцов - объектов и признаков)
df.shape

In [ ]:
# выводим типы данных
df.dtypes

In [ ]:
# описательная статистика
print(df.describe())

In [ ]:
# описательная статистика для всех (числовых) признаков
df.describe().T

In [ ]:
df.info()

In [ ]:
# имена колонок в виде списка
df.columns

In [ ]:
# проверка на отсутствующие значения для всех колонок датафрейма
df.isnull().sum()


### в ходе проверки данных обнаружилось, что в колонке Income есть пропуски, эти пропуски надо будет заполнить


In [ ]:
# посмотрел колонки на дубликаты
df.duplicated().sum()

### дубликатов не обнаружилось

In [ ]:
df.nunique()

In [ ]:
# создаём копию датафрейма для дальнейшего анализа и преобразований, чтобы не изменять исходные данные
df_eda = df.copy()

### Удаление технических колонок и заполнение пропусков Income медианой по группам

In [ ]:
# удаляем колонки Z_CostContact, Z_Revenue так как они используются для расчета затрат и  не несут полезной информации для анализа
drop_cols = ['Z_CostContact', 'Z_Revenue']
df_eda = df_eda.drop(columns=[col for col in drop_cols if col in df_eda.columns])

In [ ]:
# Заполнить медианным доходом для людей с таким же образованием и семейным положением
df['Income'] = df.groupby(['Education', 'Marital_Status'])['Income']\
                 .transform(lambda x: x.fillna(x.median()))

In [ ]:
# проверка
df.isnull().sum()

In [ ]:
# Общие траты
spending_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 
                 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df_eda['Total_Spending'] = df_eda[spending_cols].sum(axis=1)

In [ ]:
# Общие покупки
purchase_cols = ['NumDealsPurchases', 'NumWebPurchases', 
                 'NumCatalogPurchases', 'NumStorePurchases']
df_eda['Total_Purchases'] = df_eda[purchase_cols].sum(axis=1)

In [ ]:
# Средний чек
df_eda['Avg_Check'] = df_eda['Total_Spending'] / df_eda['Total_Purchases'].replace(0, np.nan)


In [ ]:
# Участие в кампаниях
campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 
                 'AcceptedCmp4', 'AcceptedCmp5']
df_eda['Total_Accepted_Cmp'] = df_eda[campaign_cols].sum(axis=1)
df_eda['Accepted_Any'] = (df_eda['Total_Accepted_Cmp'] > 0).astype(int)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_eda['Income'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='green')
plt.title('Распределение дохода', fontsize=14)
plt.xlabel('Доход')
plt.ylabel('Частота')
plt.show()

In [ ]:
# Общие траты
plt.figure(figsize=(10, 6))
plt.hist(df_eda['Total_Spending'], bins=30, edgecolor='black', alpha=0.7, color='red')
plt.title('Распределение общих трат')
plt.xlabel('Траты')
plt.ylabel('Частота')
plt.show()

In [ ]:
edu_counts = df_eda['Education'].value_counts()
plt.figure(figsize=(10, 6))
plt.bar(edu_counts.index, edu_counts.values, color='skyblue', edgecolor='black')
plt.title('Распределение образования')
plt.tick_params(axis='x', rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
marital_counts = df_eda['Marital_Status'].value_counts()
plt.bar(marital_counts.index, marital_counts.values, color='lightcoral', edgecolor='black')
plt.title('Семейное положение')
plt.tick_params(axis='x', rotation=45)
plt.show()

In [ ]:
# Средние траты по категориям
avg_spending = df_eda[spending_cols].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
bars = plt.bar(avg_spending.index, avg_spending.values, color='skyblue', edgecolor='black')
plt.title('Средние траты по категориям', fontsize=14)
plt.xlabel('Категория')
plt.ylabel('Средние траты')
plt.xticks(rotation=45)

# Добавляем значения на столбцы
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nСредние траты по категориям:")
print(avg_spending)

In [ ]:
# гистаграммы всех признаков
df.hist(figsize=(20,20))
plt.show()

In [ ]:
numeric_cols = df_eda.select_dtypes(include=[np.number]).columns
corr_matrix = df_eda[numeric_cols].corr()

# Топ корреляций с Total_Spending
if 'Total_Spending' in corr_matrix.columns:
    top_corr = corr_matrix['Total_Spending'].sort_values(ascending=False)[1:11]
    print("\nТоп-10 корреляций с общими тратами:")
    print(top_corr)

# Тепловая карта (только основные признаки)
main_features = ['Income', 'Total_Spending', 
                 'Total_Purchases', 'Recency', 'NumWebVisitsMonth', 
                  'Response']

plt.figure(figsize=(10, 8))
sns.heatmap(df_eda[main_features].corr(), 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=1)
plt.title('Корреляционная матрица основных признаков', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Сравнение Responders vs Non-responders
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Доход по Response
df_eda.boxplot(column='Income', by='Response', ax=axes[0, 0])
axes[0, 0].set_title('Доход по отклику на кампанию')
axes[0, 0].set_xlabel('Response (0=Нет, 1=Да)')

# Траты по Response
df_eda.boxplot(column='Total_Spending', by='Response', ax=axes[0, 1])
axes[0, 1].set_title('Общие траты по отклику')
axes[0, 1].set_xlabel('Response (0=Нет, 1=Да)')

# Покупки по Response
df_eda.boxplot(column='Total_Purchases', by='Response', ax=axes[1, 1])
axes[1, 1].set_title('Количество покупок по отклику')
axes[1, 1].set_xlabel('Response (0=Нет, 1=Да)')

plt.suptitle('')  # Убираем автоматический заголовок
plt.tight_layout()
plt.show()

# Статистика по группам
print("\nСравнение Responders vs Non-responders:")
response_stats = df_eda.groupby('Response')[['Income', 'Total_Spending', 
                                              'Total_Purchases']].mean()
print(response_stats)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Response по образованию
edu_response = pd.crosstab(df_eda['Education'], df_eda['Response'], normalize='index') * 100
edu_response.plot(kind='bar', stacked=True, ax=axes[0], color=['#ff9999', '#66b3ff'])
axes[0].set_title('Response по образованию (%)')
axes[0].set_ylabel('Процент')
axes[0].legend(['Нет', 'Да'])
axes[0].tick_params(axis='x', rotation=45)

# Response по семейному положению
marital_response = pd.crosstab(df_eda['Marital_Status'], df_eda['Response'], normalize='index') * 100
marital_response.plot(kind='bar', stacked=True, ax=axes[1], color=['#ff9999', '#66b3ff'])
axes[1].set_title('Response по семейному положению (%)')
axes[1].set_ylabel('Процент')
axes[1].legend(['Нет', 'Да'])
axes[1].tick_params(axis='x', rotation=45)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Boxplot для дохода
axes[0].boxplot(df_eda['Income'].dropna())
axes[0].set_title('Выбросы в Income')
axes[0].set_ylabel('Доход')

# Boxplot для трат
axes[1].boxplot(df_eda['Total_Spending'])
axes[1].set_title('Выбросы в Total Spending')
axes[1].set_ylabel('Траты')
plt.show()

# Количество выбросов по методу IQR
def count_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return len(outliers)

print("\nКоличество выбросов:")
for col in ['Income', 'Total_Spending']:
    if col in df_eda.columns:
        outliers = count_outliers(df_eda[col].dropna())
        print(f"{col}: {outliers} выбросов ({outliers/len(df_eda)*100:.1f}%)")

In [ ]:
print(f"""
КЛЮЧЕВЫЕ НАБЛЮДЕНИЯ:

ДЕМОГРАФИЯ:
- Средний доход: {df_eda['Income'].mean():.0f}
- Самые частые образования: {df_eda['Education'].mode()[0]}

ПОВЕДЕНИЕ:
- Средние траты: {df_eda['Total_Spending'].mean():.0f}
- Среднее количество покупок: {df_eda['Total_Purchases'].mean():.1f}
- Средний отклик на кампании: {df_eda['Response'].mean()*100:.1f}%

ОСНОВНЫЕ ВЫВОДЫ:
1. Клиенты с высоким доходом тратят больше
2. Семьи с детьми имеют другие паттерны покупок
""")

### Итог

В ходе EDA были выявлены основные закономерности клиентского поведения, сформированы информативные агрегированные признаки и определены ключевые факторы, влияющие на отклик на маркетинговые кампании. Подготовленные данные и обнаруженные зависимости создают основу для дальнейших этапов анализа — сегментации клиентов и построения моделей прогнозирования отклика.